# Chapter4-4 データ分析と結果の保存
## 4-4-2 データ投入

In [ ]:
import pandas as pd
iris = pd.read_csv('./input_data/iris.csv')

iris.head()

In [ ]:
iris['data_version'] = 1.0
iris.head()

In [ ]:
from sqlalchemy import create_engine

import os
pgconfig = {
    'host' : os.environ['DB_HOST']
    , 'port': os.environ['DB_PORT']
    , 'database': os.environ['DB_NAME']
    , 'user': os.environ['DB_USER']
    , 'password': os.environ['DB_PASSWORD']
}

engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig))

In [ ]:
columns = ['data_version',
'sepal_length',
'sepal_width',
'petal_length',
'petal_width' ]
iris[columns].to_sql('iris_input', engine, if_exists='append', index=False)

### 4-4-2 データの参照

In [ ]:
%load_ext sql
dsl = 'postgres://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
%sql $dsl

In [ ]:
%config SqlMagic.autopandas =True
df_input = %sql select * from iris_input
df_input.head()

## 4-4-3 分析の実行

In [ ]:
df_src = df_input[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
df_src.head()

In [ ]:
from sklearn.cluster import KMeans
kms = KMeans(n_clusters=3, random_state=1)
clusters = kms.fit_predict(df_src)
clusters

In [ ]:
from sklarn.decomposition import PCA
pca = PCA(n_components=2, random_state=1)
pca_values = pca.fit_transform(df_src)

In [ ]:
df_pca = pd.DataFrame(pca.transform(df_src), columns=['pca1', 'pca2'])
df_pca.head()

In [ ]:
df_plot = df_pca.assign(cluster_id=clusters)
df_plot.head()

In [ ]:
matplotlib inline
import seaborn as sns
sns.lmplot(x='pca1', y='pca2', data=df_plot , hue='cluster_id', markers=['o', 's', '+'], fit_reg=False)

## 4-4-4 分析結果の保存

In [ ]:
df_input.head()

In [ ]:
df_save_target = df_plot.assign(id=df_input[['id']], data_version=df_input[['data_version']])
df_save_target.head()

In [ ]:
df_save_target[['id', 'data_version', 'pca1', 'pca2', 'cluster_id' ]].to_sql('iris_cluster', engine,
if_exists='append', index=False)

In [ ]:
%sql select * from iris_cluster limit 10;